In [2]:
import requests

def load_description_cache_from_last_run():
    try:
        response = requests.get(
            "https://serlo.github.io/metadata-exports/description-cache.json",
            timeout=60,
        )

        return response.json()
    except (requests.exceptions.ReadTimeout, json.JSONDecodeError, TypeError):
        return {}
    
cache = load_description_cache_from_last_run()

In [3]:
import json

import sys
import os

sys.path.append(os.path.join(os.getcwd(), '..'))

from utils import has_description

for key, value in cache.items():
    if not has_description(value):
        print(json.dumps({"key": key, "value": value}, indent=2))

{
  "key": "https://serlo.org/8057",
  "value": {
    "description": null,
    "version": {
      "id": "https://serlo.org/283805"
    }
  }
}
{
  "key": "https://serlo.org/18582",
  "value": {
    "description": null,
    "version": {
      "id": "https://serlo.org/259901"
    }
  }
}
{
  "key": "https://serlo.org/21603",
  "value": {
    "description": null,
    "version": {
      "id": "https://serlo.org/187349"
    }
  }
}
{
  "key": "https://serlo.org/21606",
  "value": {
    "description": null,
    "version": {
      "id": "https://serlo.org/200727"
    }
  }
}
{
  "key": "https://serlo.org/22363",
  "value": {
    "description": null,
    "version": {
      "id": "https://serlo.org/187344"
    }
  }
}
{
  "key": "https://serlo.org/22366",
  "value": {
    "description": null,
    "version": {
      "id": "https://serlo.org/187342"
    }
  }
}
{
  "key": "https://serlo.org/22369",
  "value": {
    "description": null,
    "version": {
      "id": "https://serlo.org/253993"
    }

In [4]:
response = requests.get(
    "https://serlo.github.io/metadata-exports/serlo-metadata.json",
    timeout=60,
)

metadata = response.json()

In [7]:
record = next(f for f in metadata if f["id"] == "https://serlo.org/267014")
record

{'about': [{'type': 'Concept',
   'id': 'http://w3id.org/kim/schulfaecher/s1017',
   'inScheme': {'id': 'http://w3id.org/kim/schulfaecher/'}}],
 '@context': ['https://w3id.org/kim/amb/context.jsonld',
  {'@language': 'de',
   '@vocab': 'http://schema.org/',
   'id': '@id',
   'type': '@type'}],
 'id': 'https://serlo.org/267014',
 'type': ['LearningResource', 'Quiz'],
 'dateCreated': '2023-02-03T16:03:34+00:00',
 'dateModified': '2023-03-03T22:07:17+00:00',
 'creator': [{'type': 'Person',
   'id': 'https://serlo.org/87602',
   'name': 'Karin',
   'affiliation': {'id': 'https://serlo.org/organization',
    'name': 'Serlo Education e.V.',
    'type': 'Organization'}},
  {'type': 'Person',
   'id': 'https://serlo.org/169563',
   'name': 'Peter',
   'affiliation': {'id': 'https://serlo.org/organization',
    'name': 'Serlo Education e.V.',
    'type': 'Organization'}}],
 'identifier': {'propertyID': 'UUID',
  'type': 'PropertyValue',
  'value': 267014},
 'image': 'https://de.serlo.org/_asse

In [17]:
import re

from utils import pick
from serlo_api_client import fetch_content

def get_description_from_content(record):
    identifier = pick(["identifier", "value"], record)

    if not isinstance(identifier, int):
        return None

    content_raw = fetch_content(identifier)

    if not isinstance(content_raw, str):
        return None

    try:
        content = json.loads(content_raw)

        print(content)
        
        content_text = get_text(content)

        print(content_text)
        
        first_paragraph = content_text.split("\n", maxsplit=1)[0]

        print(first_paragraph)
        
        if first_paragraph.isspace():
            return None

        return re.sub(" +", " ", first_paragraph)
    except json.JSONDecodeError:
        return None


def get_text(data) -> str:
    if isinstance(data, list):
        return "".join(get_text(child) for child in data)
    if isinstance(data, dict):
        return get_text_from_dict(data)
    return ""


def get_text_from_dict(data):
    if "plugin" in data and data["plugin"] == "text" and "state" in data:
        return get_text(data["state"]) + "\n"
    if "type" in data and data["type"] == "h" and "children" in data:
        return get_text(data["children"]) + "\n"
    if "type" in data and data["type"] == "p" and "children" in data:
        return get_text(data["children"]) + " "
    if "type" in data and data["type"] == "math" and "src" in data:
        return "$" + data["src"] + "$"
    if "text" in data and isinstance(data["text"], str):
        return data["text"]
    return "".join(get_text(child) for child in data.values())

get_description_from_content(record)

{'plugin': 'exercise', 'state': {'content': {'plugin': 'rows', 'state': [{'plugin': 'injection', 'state': '/266939'}]}}}




''

In [19]:
print("   ".isspace())

True
